# Colab A100 クイックスタート

- ランタイム: GPU（A100 推奨）に切替
- 依存をインストールし、A100最適化の既定で実行
- 保存: first_seen.csv / stats.csv を逐次保存（`--autosave-sec`）
- さらに高速: `accelerated_counter.py` のバッチ推論（追跡OFF）を利用



In [ ]:
%cd /content/colab-turbo-counter
pip -q install ultralytics torch torchvision opencv-python-headless shapely pandas tqdm deepface gspread oauth2client onnxruntime-gpu


In [ ]:
python colab/turbo_counter.py \
  --video "/content/drive/MyDrive/アオハル祭/input_videos/merged_20250816_1141-1951.mkv" \
  --outdir "out_20250816" \
  --cam-id 20250816 \
  --device cuda --progress --progress-sec 2 --autosave-sec 60


In [ ]:
python colab/accelerated_counter.py \
  --video "/content/drive/MyDrive/アオハル祭/input_videos/merged_20250816_1141-1951.mkv" \
  --outdir "out_20250816_ultralite" \
  --cam-id 20250816 \
  --device cuda --mode ultralite --half --no-tracking \
  --batch 128 --imgsz 320 --vid-stride 5 --max-det 80 \
  --progress --progress-sec 2 --autosave-sec 60


In [ ]:
python colab/accelerated_counter.py \
  --video "/content/drive/MyDrive/アオハル祭/input_videos/merged_20250816_1141-1951.mkv" \
  --outdir "out_20250816_track" \
  --cam-id 20250816 \
  --device cuda --mode fast --half \
  --progress --progress-sec 2 --autosave-sec 60


In [ ]:
python colab/accelerated_counter.py \
  --video "/content/drive/MyDrive/アオハル祭/input_videos/merged_20250816_1141-1951.mkv" \
  --outdir "out_20250816_onnx" \
  --cam-id 20250816 \
  --device cuda --mode ultrafast --half --no-tracking --onnx \
  --batch 128 --imgsz 416 --vid-stride 4 --max-det 120 \
  --progress --progress-sec 2


In [ ]:
python colab/show_preview.py --video "/content/drive/MyDrive/アオハル祭/input_videos/merged_20250816_1141-1951.mkv" --step 120 --limit 3


## 複数日の結果をマージ（重複統合）
- 入力: 各出力フォルダ内の `first_seen.csv` と任意の `turbo_embeddings.npz`
- 出力: `merged_first_seen.csv`, `mapping.csv`, `clusters.csv`, `stats.csv`
- しきい値: `--alias-threshold`（既定0.92、上げるほど安全側）


In [ ]:
python colab/merge_turbo.py \
  --inputs \
  "/content/drive/MyDrive/アオハル祭/out_20250816_ultralite" \
  "/content/drive/MyDrive/アオハル祭/out_20250817_ultralite" \
  --outdir "/content/drive/MyDrive/アオハル祭/merged_all" \
  --alias-threshold 0.92


In [ ]:
# Colab Ultra-Fast People Counter (Turbo)

GUIなし、オーバーレイなし。初回観測のみをCSV出力します。


In [ ]:
!nvidia-smi || echo 'No GPU'
!python -V
!pip -q install ultralytics torch torchvision opencv-python-headless shapely pandas tqdm


In [ ]:
# 左ペイン(ファイル)で `colab/turbo_counter.py` をアップロードしてください。
# または、下のセルでGoogle Driveをマウントして直接参照することも可能です。
print('Upload colab/turbo_counter.py via left panel or mount Drive to use existing file.')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

video16 = '/content/drive/MyDrive/merged_20250816.mp4'
out16 = '/content/turbo_20250816'

!python colab/turbo_counter.py --video "$video16" --outdir "$out16" --cam-id 20250816 \
  --model yolov8n.pt --device auto --conf 0.5 --imgsz 640 --vid-stride 2 --max-det 200 --progress --progress-sec 2
